In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
import os,json,openai; openai.api_key = os.environ['OPENAI_API_KEY']
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType
from langchain.agents import load_tools, initialize_agent
from langchain.agents import tool


In [3]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

In [4]:
# import langchain; langchain.debug = True
# langchain.debug = False

In [5]:
class Nutrition(BaseModel):
    ingredient_name: str = Field(description='Ingredient name (kcal/100g)')
    calories: int = Field(description='calories (kcal/100g)')
    carbohydrates: float = Field(description='carbohydrates (g/100g)')
    protein: float = Field(description='protein (g/100g)')
    fat: float = Field(description='fat (g/100g)')

    vitamin_c: float = Field(description='vitamin_c (mg/100g)')
    fiber: float = Field(description='fiber (g/100g)')

prompt_template_food = """
Please tell me the nutrition data of {food_name}
{format_instructions}
"""

prompt_food = ChatPromptTemplate.from_template(template=prompt_template_food)
chat_nutrition = ChatOpenAI(temperature=0.0)
nutrition_parser = PydanticOutputParser(pydantic_object=Nutrition)

@tool
def get_ingredient_nutrition(food_name: str) -> dict:
    """食材・調味料の栄養データを返す。\
    入力は常に一つの食材・調味料名であり、出力は常にその食材・調味料の栄養データである。\
    レシピ全体の栄養計算は常にこの関数の外で行われる。
    """
    messages = prompt_food.format_messages(
        food_name=food_name,
        format_instructions=nutrition_parser.get_format_instructions()
    )
    response = chat_nutrition(messages)
    food_data = nutrition_parser.parse(response.content)
    return json.dumps(food_data.dict(),indent=2,ensure_ascii=False)

In [6]:
# data = get_ingredient_nutrition('白菜')
# print(data)

In [7]:
prompt_template = "{food_name}を使ったレシピを一つ教えてください。\
材料の分量は一人前にし、全てグラムで出力してください。"
prompt_recipe = ChatPromptTemplate.from_template(template=prompt_template)
chat_recipe = ChatOpenAI(temperature=0.5)

@tool
def recommend_recipe(food_name: str) -> dict:
    """入力の食材を使ったレシピを一つ返す。\
    入力は常に食材名であるべきで、出力は常に一つのレシピである。
    """
    messages = prompt_recipe.format_messages(food_name=food_name)
    response = chat_recipe(messages)
    return response.content

In [8]:
# recipe = recommend_recipe('白菜')
# print(recipe)

In [9]:
llm = ChatOpenAI(temperature=0)
tools = load_tools(["llm-math"], llm=llm)

In [10]:
agent= initialize_agent(
    tools + [recommend_recipe,get_ingredient_nutrition], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [11]:
agent("豚バラを使ったレシピを一つ提案し、そのレシピ全体の栄養成分を教えてください。")



> Entering new  chain...


Thought: I can use the `recommend_recipe` tool to find a recipe that uses pork belly. Then, I can use the `get_ingredient_nutrition` tool to get the nutrition information for each ingredient in the recipe. Finally, I can calculate the total nutrition for the entire recipe.

Action:
```
{
  "action": "recommend_recipe",
  "action_input": "豚バラ"
}
```
Observation: 豚バラのしょうが焼き

材料:
- 豚バラ肉: 200g
- しょうが: 10g
- にんにく: 5g
- しょうゆ: 20ml
- みりん: 20ml
- 砂糖: 10g
- サラダ油: 適量

作り方:
1. 豚バラ肉を1cm厚さに切ります。
2. しょうがとにんにくをみじん切りにします。
3. 豚バラ肉にしょうがとにんにくをまぶし、しょうゆ、みりん、砂糖を混ぜたタレに漬け込みます。タレが全体に行き渡るようによく揉み込みます。
4. フライパンにサラダ油を熱し、豚バラ肉を並べて焼きます。両面がこんがりと焼けるまで約5分ずつ焼きます。
5. 豚バラ肉を取り出し、余分な油をキッチンペーパーで拭き取ります。
6. お皿に盛り付けて、お好みでごまや青ネギをトッピングして完成です。

お楽しみください！
Thought:Now that I have the recipe for "豚バラのしょうが焼き", I can use the `get_ingredient_nutrition` tool to get the nutrition information for each ingredient in the recipe. Then, I can calculate the total nutrition for the entire recipe.

Action:
```
{
  "action": "get_ingredient_nutriti

{'input': '豚バラを使ったレシピを一つ提案し、そのレシピ全体の栄養成分を教えてください。',
 'output': 'The total nutrition for the entire recipe is 2191 calories.'}